In [ ]:
import torch 
import matplotlib.pyplot as plt 
#from lanczos import lanczos 
#torch.set_num_threads(1)
torch.manual_seed(42)

In [ ]:
d = 2 # fixed
D = 20
dtype = torch.float64
#Lancsize = 100 

B = 0.01* torch.randn(d, D, D, dtype=dtype)
#symmetrize initial boundary PEPS
B = (B + B.permute(0, 2, 1))/2. #symmetrize
A = torch.nn.Parameter(B.view(d, D**2))

#dimer covering
T = torch.zeros(d, d, d, d, dtype=dtype)
T[0, 0, 0, 1] = 1.0 
T[0, 0, 1, 0] = 1.0 
T[0, 1, 0, 0] = 1.0 
T[1, 0, 0, 0] = 1.0 
T = T.view(d, d**2, d)

optimizer = torch.optim.LBFGS([A], max_iter=20, tolerance_grad=0, tolerance_change=0, line_search_fn="strong_wolfe")

In [ ]:
def symmetrize(A):
    Asymm = A.view(d, D, D) 
    Asymm = (Asymm + Asymm.permute(0, 2, 1))/2
    return Asymm.view(d, D**2)

def closure():
    optimizer.zero_grad()
    Asymm = symmetrize(A)
    T1 = torch.einsum('xa,xby,yc',(Asymm,T,Asymm)).view(D,D,d,d,D,D).permute(0,2,4,1,3,5).contiguous().view(D**2*d, D**2*d)
    T2 = torch.einsum('xa,xb',(Asymm,Asymm)).view(D,D,D,D).permute(0,2,1,3).contiguous().view(D**2,D**2)

    #phi0 = torch.randn(D**2*d)
    #phi0 = phi0/phi0.norm()
    #eigval_1, _ = lanczos(lambda x: T1@x, phi0, Lancsize)

    #phi0 = torch.randn(D**2)
    #phi0 = phi0/phi0.norm()
    #eigval_2, _ = lanczos(lambda x: T2@x, phi0, Lancsize)

    eigval_1,_ = torch.symeig(T1,eigenvectors=True)
    eigval_2,_ = torch.symeig(T2,eigenvectors=True)

    lnT = torch.log(eigval_1.max())
    lnZ = torch.log(eigval_2.max())
    loss = -lnT + lnZ
    loss.backward()
    return loss

In [ ]:
exact = 0.2915609040 
Nepoch = 100 
F=[]
for epoch in range(Nepoch):
    loss = optimizer.step(closure) # closure is used to calculate the loss
    F.append(-loss.item())
    print ('epoch, residual entropy', epoch, -loss.item(), (-loss.item()-exact)/exact)
plt.plot(range(Nepoch),F)
plt.show()